<a href="https://colab.research.google.com/github/apadhikary/ERA_V2_New/blob/main/S11/Session11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load Modules
!git clone https://github.com/apadhikary/ERA_V2_New.git
print("Loaded Modules and Models")

Cloning into 'ERA_V2_New'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 40 (delta 19), reused 23 (delta 8), pack-reused 0
Receiving objects: 100% (40/40), 11.84 KiB | 5.92 MiB/s, done.
Resolving deltas: 100% (19/19), done.
Loaded Modules and Models


In [ ]:
!pip install torch-lr-finder -q --quiet
!pip install grad-cam
!pip install lightning-bolts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.0-py3-none-any.whl size=38071 sha256=855af4ce3e0e52a244749f1e8019294c1c423cf5fc4fe2e410d2afb4139e1144
  Stored in directory: /root/.cache/pip/wheels/5b/e5/3d/8548241d5cffe53ad1476c566a61ad9bf09cc61a9430f09726
Successfully built grad-cam
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.8/300.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 49.5 MB/s eta 0:00:00


### Import Modules

In [ ]:
from __future__ import print_function
# For inline matplotlib plotting
%matplotlib inline
from ERA_V2_New.S11.models import *
from ERA_V2_New.S11.utils import *
from ERA_V2_New.S11.main import *

### Load Train and Test Dataloader

In [ ]:
# Load Train and Test Dataloader
cifar10 = dataset_cifar10(batch_size = 512)
trainloader, testloader = cifar10.train_test_datasets()

CUDA Available? False


100%|██████████| 170498071/170498071 [00:01<00:00, 103001228.63it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### Plot Sample Pictures

In [ ]:
cifar10.sample_pictures(trainloader, train_flag = True)
classes = cifar10.classes

<Figure size 2500x2000 with 0 Axes>

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

### Loading Custom Model

In [ ]:
# Model
from ERA_V2_New.S11.models.resnet import *

# Loading Custom Model
net = ResNet18()

# Using cuda if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Sending network to device
model = net.to(device)

In [ ]:
# Model Summary
!pip install torchsummary
from torchsummary import summary

# Model Summary
summary(model, input_size=(3, 32, 32))

### LR Finder

In [ ]:
import copy
import torch
from torch_lr_finder import LRFinder
import torch.nn as nn
import torch.optim as optim
import numpy as np

model_copy = copy.deepcopy(model)
optimizer = optim.SGD(model_copy.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()
lr_finder = LRFinder(model_copy, optimizer, criterion, device="cuda")
lr_finder.range_test(trainloader, end_lr=10, num_iter=200, step_mode="exp")
lr_finder.plot(log_lr = False) # to inspect the loss-learning rate graph
lr_finder.reset() # to reset the model and optimizer to their initial state

### Training and Testing

In [ ]:
from torch.optim.lr_scheduler import OneCycleLR
from ERA_V2_New.S11.main import train, test
train_losses = []
train_accuracy = []
test_losses = []
test_accuracy = []
learning_rates = []
EPOCHS = 20
train_model = train()
test_model = test()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = OneCycleLR(optimizer, max_lr = 0.001, steps_per_epoch = len(trainloader),
                        epochs = EPOCHS, pct_start = 0.2, div_factor = 100,
                        three_phase = False, final_div_factor = 100,
                        anneal_strategy = 'linear')
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train_loss, correct, processed = train_model.train(model, device, trainloader, optimizer, epoch)
    train_losses.append(train_loss)
    train_accuracy.append(100*correct/processed)
    scheduler.step()
    # learning_rates.append(get_lr(optimizer))
    test_loss, correct = test_model.test(model, device, testloader)
    test_losses.append(test_loss)
    test_accuracy.append(100*correct/len(testloader))




### Plot Training and Validation Curves

In [ ]:
from ERA_V2_New.S11.utils import plot_loss_accuracy_graph, plot_loss_accuracy_graph_OneCLR
plot_loss_accuracy_graph(train_model, test_model, EPOCHS)

In [ ]:
plot_loss_accuracy_graph_OneCLR(train_accuracy, train_losses, test_accuracy, test_losses)

### Plot Learning Rates

In [ ]:
import matplotlib.pyplot as plt
plt.plot(learning_rates)
plt.title("Training Learning Rate")

### Save The Model

In [ ]:
torch.save(model.state_dict(), "model.pth")
model.load_state_dict(torch.load("model.pth"), strict=False)

### Display Sample Test Images

In [ ]:
cifar10.sample_pictures(testloader, train_flag = False)
classes = cifar10.classes

### Evaluating Training Accuracy

In [ ]:
from ERA_V2_New.S11.utils import calAccuracy
calAccuracy(model, trainloader, device)

### Evaluating Test Accuracy

In [ ]:
calAccuracy(model, testloader, device)

### Evaluating Test Accuracy Over Different Classes

In [ ]:
from ERA_V2_New.S11.utils import calClassAccuracy
calClassAccuracy(model, testloader, classes, device)

### Plot Misclassified Images

In [ ]:
from ERA_V2_New.S11.utils import plot_misclassified_images
plot_misclassified_images(model, testloader, classes, device)

### Plot Grad-CAM Images

In [ ]:
from ERA_V2_New.S11.utils import plot_grad_cam_images
plot_grad_cam_images(model, testloader, classes, device)

### Display Confusion Matrix

In [ ]:
from ERA_V2_New.S11.utils import visualize_confusion_matrix

visualize_confusion_matrix(classes, device, model, testloader)